In [1]:
# Crear el notebook completo en un archivo .ipynb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-12-01']
test_data = df_unificado[df_unificado['Subproceso_Costo'] == '2022-12-01']

In [2]:
# Modelo 1: Linear Regression (Múltiple)
resultados_lr = {}
lr_model = LinearRegression()
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        lr_model.fit(X_train, y_train)
        y_pred_2022 = lr_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_lr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_lr_df = pd.DataFrame(resultados_lr).T
print("Resultados para Linear Regression (Múltiple):")
print(resultados_lr_df)

Resultados para Linear Regression (Múltiple):
                                        Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1458964.483104   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR           174017.429299    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR                 42167.44486      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE              -427661.669636    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                 68247.810665      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR            76814.537505     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA             -8878.207824      1686.65384   
ADMINISTRACIÓN MINA                       -41572.482979         -720.11   
CARGUÍO                                 -1246171.548652    787927.53518   
CARGUÍO ENCUENTRO                          -15346.29385       195.03104   
CARGUÍO ESPERANZA                        3164530.189899    290768.67368   
CARGUÍO ESPERANZA SUR                     653556.80978

In [3]:
# Modelo 2: Polynomial Regression (Regresión Polinómica)
resultados_pr = {}
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        
        pr_model = LinearRegression()
        pr_model.fit(X_train_poly, y_train)
        y_pred_2022 = pr_model.predict(X_test_poly)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_pr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_pr_df = pd.DataFrame(resultados_pr).T
print("Resultados para Polynomial Regression (Regresión Polinómica):")
print(resultados_pr_df)

Resultados para Polynomial Regression (Regresión Polinómica):
                                        Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1231323.520931   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR           260771.989637    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR                45653.007519      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE              -247463.756752    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                135555.989098      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR           130068.612646     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA            -95565.501193      1686.65384   
ADMINISTRACIÓN MINA                       -89731.330989         -720.11   
CARGUÍO                                 -1264793.169146    787927.53518   
CARGUÍO ENCUENTRO                         -12580.769596       195.03104   
CARGUÍO ESPERANZA                        2929107.964158    290768.67368   
CARGUÍO ESPERANZA SUR                 

In [4]:
# Modelo 3: Neural Network (Redes Neuronales)
resultados_nn = {}
nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        nn_model.fit(X_train, y_train)
        y_pred_2022 = nn_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_nn[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_nn_df = pd.DataFrame(resultados_nn).T
print("Resultados para Neural Network (Redes Neuronales):")
print(resultados_nn_df)

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum it

Resultados para Neural Network (Redes Neuronales):
                                       Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP                 130.014637   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR           92335.202529    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR               18056.704724      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE                9050.133735    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                24340.101524      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR           79608.998089     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA            40964.862304      1686.65384   
ADMINISTRACIÓN MINA                       65049.405507         -720.11   
CARGUÍO                                    -102.406119    787927.53518   
CARGUÍO ENCUENTRO                          5327.979217       195.03104   
CARGUÍO ESPERANZA                         44442.645199    290768.67368   
CARGUÍO ESPERANZA SUR                    197613.490323    660

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
# Modelo 4: Random Forest
resultados_rf = {}
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        rf_model.fit(X_train, y_train)
        y_pred_2022 = rf_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_rf[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_rf_df = pd.DataFrame(resultados_rf).T
print("Resultados para Random Forest:")
print(resultados_rf_df)

Resultados para Random Forest:
                                        Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP               1885089.91181   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR           258452.113812    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR                69774.290918      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE                 8632.135279    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                120846.524176      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR           136909.446266     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA              -397.900905      1686.65384   
ADMINISTRACIÓN MINA                        -4645.027969         -720.11   
CARGUÍO                                 -1294891.403305    787927.53518   
CARGUÍO ENCUENTRO                            134.612774       195.03104   
CARGUÍO ESPERANZA                        3272010.934508    290768.67368   
CARGUÍO ESPERANZA SUR                     826736.558102    660113.019

In [6]:
# Modelo 5: Gradient Boosting
resultados_gb = {}
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        gb_model.fit(X_train, y_train)
        y_pred_2022 = gb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_gb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_gb_df = pd.DataFrame(resultados_gb).T
print("Resultados para Gradient Boosting:")
print(resultados_gb_df)

Resultados para Gradient Boosting:
                                        Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1744019.553376   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR           255846.008416    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR                64836.842511      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE                10983.852478    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                121732.079542      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR           189446.548599     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA              -317.105289      1686.65384   
ADMINISTRACIÓN MINA                        -4270.491691         -720.11   
CARGUÍO                                   -1569579.8183    787927.53518   
CARGUÍO ENCUENTRO                            -77.068137       195.03104   
CARGUÍO ESPERANZA                        3266254.365425    290768.67368   
CARGUÍO ESPERANZA SUR                     788227.335403    660113

In [7]:
# Modelo 6: XGBoost
resultados_xgb = {}
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        xgb_model.fit(X_train, y_train)
        y_pred_2022 = xgb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-12-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_xgb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_xgb_df = pd.DataFrame(resultados_xgb).T
print("Resultados para XGBoost:")
print(resultados_xgb_df)

Resultados para XGBoost:
                                       Predicción 2022 Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP                1794304.625   1801845.02384   
ADM OPERACIÓN Y GESTIÓN ESP SUR            1107.853638    282766.49464   
ADM OPERACIÓN Y GESTIÓN MIR                 69217.1875      52742.7824   
ADM OPERACIÓN Y GESTIÓN OXE               61295.339844    157301.84704   
ADM OPERACIÓN Y GESTIÓN TC                112400.53125      4345.54392   
ADM OPERACIÓN Y GESTIÓN TES SUR           153922.90625     193648.0008   
ADMINISTRACIÓN DESARROLLO MINA              208.432358      1686.65384   
ADMINISTRACIÓN MINA                       -8098.395508         -720.11   
CARGUÍO                                     -1594447.0    787927.53518   
CARGUÍO ENCUENTRO                           135.229187       195.03104   
CARGUÍO ESPERANZA                           3363435.25    290768.67368   
CARGUÍO ESPERANZA SUR                      543500.8125    660113.01992   
CARGUÍO MIRAD

In [8]:
# Comparación de resultados
resultados_totales = {
    'Linear Regression (Múltiple)': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}
for modelo, resultados in resultados_totales.items():
    print(f"Resumen para {modelo}:")
    print(resultados[['Diferencia %']])
    print("\n\n")

Resumen para Linear Regression (Múltiple):
                                       Diferencia %
ADM OPERACIÓN Y GESTIÓN ESP              -19.029414
ADM OPERACIÓN Y GESTIÓN ESP SUR          -38.458964
ADM OPERACIÓN Y GESTIÓN MIR              -20.050777
ADM OPERACIÓN Y GESTIÓN OXE             -371.873266
ADM OPERACIÓN Y GESTIÓN TC              1470.524011
ADM OPERACIÓN Y GESTIÓN TES SUR          -60.332904
ADMINISTRACIÓN DESARROLLO MINA          -626.379961
ADMINISTRACIÓN MINA                     5673.073972
CARGUÍO                                 -258.158142
CARGUÍO ENCUENTRO                      -7968.641756
CARGUÍO ESPERANZA                        988.332574
CARGUÍO ESPERANZA SUR                     -0.993195
CARGUÍO MIRADOR                            3.450473
CARGUÍO TESORO CENTRAL                   484.922467
CARGUÍO TESORO SUR                        60.300532
EQUIPOS  AUXILIARES                       -422.3225
EQUIPOS DE SOPORTE (APOYO)               -52.123531
MANTENIMIENTO MINA   

In [9]:
import pandas as pd
import numpy as np

# Resultados de los modelos (simula los DataFrames que imprimiste anteriormente)
resultados = {
    'Linear Regression': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}

# Crear un DataFrame para almacenar los mejores modelos para cada subproceso
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mejor Modelo', 'Diferencia %'])

# Comparar los resultados para cada subproceso
for subproceso in resultados['Linear Regression'].index:
    mejor_modelo = None
    menor_diferencia = np.inf
    
    for modelo, df in resultados.items():
        if subproceso in df.index and df.loc[subproceso, 'Diferencia %'] != 'N/A':
            diferencia = abs(df.loc[subproceso, 'Diferencia %'])
            if diferencia < menor_diferencia:
                menor_diferencia = diferencia
                mejor_modelo = modelo
    
    # Guardar el mejor modelo para el subproceso
    nueva_fila = pd.DataFrame({
        'Subproceso': [subproceso],
        'Mejor Modelo': [mejor_modelo],
        'Diferencia %': [menor_diferencia]
    })
    
    mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)
    
# Exportar como Excel los resultados
mejor_modelo_df.to_excel('C:/Users/Administrator/Documents/GitHub/Hackathon2-2024/Desarollo/Resultados/Costos_Indicadores_Mina.xlsx', index=False)

# Mostrar los resultados
mejor_modelo_df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20396\3902238505.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)


,Subproceso,Mejor Modelo,Diferencia %
0,ADM OPERACIÓN Y GESTIÓN ESP,XGBoost,0.418482
1,ADM OPERACIÓN Y GESTIÓN ESP SUR,Polynomial Regression,7.778328
2,ADM OPERACIÓN Y GESTIÓN MIR,Polynomial Regression,13.442171
3,ADM OPERACIÓN Y GESTIÓN OXE,XGBoost,61.033299
4,ADM OPERACIÓN Y GESTIÓN TC,Neural Network,460.116339
5,ADM OPERACIÓN Y GESTIÓN TES SUR,Gradient Boosting,2.169634
6,ADMINISTRACIÓN DESARROLLO MINA,XGBoost,87.642256
7,ADMINISTRACIÓN MINA,Gradient Boosting,493.033244
8,CARGUÍO,Neural Network,100.012997
9,CARGUÍO ENCUENTRO,XGBoost,30.662736
